In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas import Series,DataFrame
from lmfit import Model
from numpy import exp

In [2]:
def gompertz(t,A,mu,lamb):
    y = A*exp(-exp(mu*exp(1)/A*(lamb-t)+1))
    return y

In [3]:
gomp_mod = Model(gompertz)

In [4]:
data = pd.ExcelFile('ART_coverage2.xlsx')
dframe = data.parse('Data')
dframe2 = dframe.pivot_table(index='Country Name',columns='Year',values='Value')
dframe2 = dframe2.dropna(axis=0,how='all')
dframe2 = dframe2.dropna(axis=1,how='all')
diff = dframe2.diff(axis=1)
diff.fillna(0,inplace=True)

In [5]:
t = np.arange(0,15)
y = np.asarray(dframe2.ix['Argentina'])

In [6]:
params = gomp_mod.make_params()

In [7]:
gomp_mod.set_param_hint('A',value=dframe2.ix['Argentina'].max(),min=0,max=100)
gomp_mod.set_param_hint('mu',value=diff.ix['Argentina'].max())
gomp_mod.set_param_hint('lamb',value=0)

In [8]:
result = gomp_mod.fit(y,t=t)

In [9]:
print(result.fit_report())

[[Model]]
    Model(gompertz)
[[Fit Statistics]]
    # function evals   = 68
    # data points      = 15
    # variables        = 3
    chi-square         = 61.155
    reduced chi-square = 5.096
    Akaike info crit   = 30.428
    Bayesian info crit = 32.552
[[Variables]]
    A:      73.1281530 +/- 16.49290 (22.55%) (init= 58.75)
    mu:     2.31238112 +/- 0.544336 (23.54%) (init= 7.5)
    lamb:  -12.7661292 +/- 3.546308 (27.78%) (init= 0)
[[Correlations]] (unreported correlations are <  0.100)
    C(mu, lamb)                  =  0.993 
    C(A, mu)                     = -0.947 
    C(A, lamb)                   = -0.916 



In [10]:
result.bic

32.551674679518385

In [11]:
y=dframe2.ix['Argentina']
plt.plot(t, y,         'bo')
plt.plot(t, result.init_fit, 'k--')
plt.plot(t, result.best_fit, 'r-')
plt.show()

In [12]:
### Initialize dictionary
gomp_dict ={}

In [13]:
### Iterate model build for each country
for i in range(0,len(dframe2)):
    ### Select country
    country = dframe2.index[i]
    ### Select coverage values
    y = np.asarray(dframe2.ix[country])
    ### Set parameter hints
    gomp_mod.set_param_hint('A',value=dframe2.ix[country].max(),min=0,max=100)
    gomp_mod.set_param_hint('mu',value=diff.ix[country].max())
    gomp_mod.set_param_hint('lamb',value=0)
    ### Fit model
    result = gomp_mod.fit(y,t=t,verbose=False)
    ### Store fitted model in dictionary
    gomp_dict[country] = result

In [14]:
def logistic(t,A,mu,lamb):
    y = A/(1+exp((4*mu/A)*(lamb-t)+2))
    return y

In [15]:
logmod = Model(logistic)

In [16]:
### Initialize dictionary
log_dict ={}

In [17]:
### Iterate model build for each country
for i in range(0,len(dframe2)):
    ### Select country
    country = dframe2.index[i]
    ### Select coverage values
    y = np.asarray(dframe2.ix[country])
    ### Set parameter hints
    logmod.set_param_hint('A',value=dframe2.ix[country].max(),min=0,max=100)
    logmod.set_param_hint('mu',value=diff.ix[country].max())
    logmod.set_param_hint('lamb',value=0)
    ### Fit model
    result = logmod.fit(y,t=t,verbose=False)
    ### Store fitted model in dictionary
    log_dict[country] = result

In [19]:
import os

In [20]:
subsahara=['Angola','Benin','Botswana','Burkina Faso','Burundi','Cameroon',
           'Cabo Verde','Central African Republic','Chad','Cote d\'Ivoire',
          'Eritrea','Gabon','Ghana','Guinea','Guinea-Bissau','Kenya',
          'Lesotho','Liberia','Madagascar','Malawi','Mali','Mauritania','Mauritius',
          'Mozambique','Namibia','Niger','Nigeria','Rwanda','Sao Tome and Principe',
          'Senegal','Sierra Leone','Somalia','South Africa',
          'South Sudan','Sudan','Swaziland','Tanzania','Togo','Uganda','Zambia',
          'Zimbabwe','Equatorial Guinea','Ethiopia','Congo, Rep.','Congo, Dem. Rep.']

In [57]:
country='South Sudan'
axis_font = {'size':'16'}
t2=np.linspace(0,20,100)
y=dframe2.ix[country]
fig=plt.figure()
plt.plot(t, y, 'bs',label='Data')
plt.plot(t2, gomp_dict[country].eval(t=t2), 'r',label='Gompertz')
plt.plot(t2, log_dict[country].eval(t=t2), 'g--',label='Logistic')
plt.axis(x=np.linspace(0,20,1))
plt.ylabel('ART coverage (%)',**axis_font)
plt.xlabel('Time (in years since 2000)',**axis_font)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc='upper left',bbox_to_anchor=(1,0.5))
plt.show()

In [45]:
gomp_dict[country].eval(t=t2)

array([  0.0930189 ,   0.11934429,   0.15176373,   0.19134104,
         0.23925177,   0.29678185,   0.36532414,   0.44637262,
         0.54151425,   0.6524186 ,   0.78082522,   0.92852913,
         1.09736448,   1.28918675,   1.50585385,   1.74920639,
         2.02104752,   2.32312279,   2.65710026,   3.0245514 ,
         3.42693294,   3.86557015,   4.34164162,   4.85616598,
         5.40999052,   6.00378206,   6.63801995,   7.31299135,
         8.02878876,   8.78530974,   9.58225874,  10.4191509 ,
        11.2953177 ,  12.20991429,  13.16192828,  14.15018987,
        15.17338303,  16.23005758,  17.318642  ,  18.43745669,
        19.5847275 ,  20.75859951,  21.9571507 ,  23.17840541,
        24.42034759,  25.68093352,  26.95810409,  28.24979644,
        29.55395485,  30.86854103,  32.19154355,  33.52098645,
        34.85493714,  36.19151336,  37.52888944,  38.86530167,
        40.19905296,  41.52851674,  42.8521401 ,  44.16844633,
        45.47603676,  46.77359203,  48.0598728 ,  49.33

In [20]:
for country in subsahara:
    t2=np.linspace(0,21,100)
    y=dframe2.ix[country]
    fig=plt.figure()
    plt.plot(t, y,         'bo')
    plt.plot(t2, gomp_dict[country].eval(t=t2), 'r-')
    plt.plot(t2, log_dict[country].eval(t=t2), 'g--')
    plt.title(country)
    plt.axis(x=np.arange(0,20))
    plt.ylabel('ART coverage (%)')
    plt.xlabel('Time (in years since 2000)')
    filename=country+'.png'
    plt.savefig(filename)

C:\Users\Ben\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [21]:
#### Countries to be fixed
### Gompertz - Benin, Chad, Senegal, Somalia, Sudan
### Logistic - Sao Tome and Principe, Sudan, Uganda
fixed_gomp = {}
bad_fits_gomp = ['Benin','Chad','Senegal','Somalia','Sudan']
t = np.arange(0,15)
for country in bad_fits_gomp:
    y = np.asarray(dframe2.ix[country])
    gomp_mod.set_param_hint('A',value=dframe2.ix[country].max(),min=0,max=100)
    gomp_mod.set_param_hint('mu',value=diff.ix[country].max(),min=0)
    gomp_mod.set_param_hint('lamb',value=0)
    result = gomp_mod.fit(y,t=t,verbose=False)
    fixed_gomp[country]=result

In [22]:
### Fixed: Chad, Somalia
gomp_dict['Chad'] = fixed_gomp['Chad']
gomp_dict['Somalia'] = fixed_gomp['Somalia']

In [23]:
#### Benin
country='Benin'
y = np.asarray(dframe2.ix[country])
### Set hints from successful logistic fit
gomp_mod.set_param_hint('A',value=46,min=0,max=100)
gomp_mod.set_param_hint('mu',value=15,min=0)
gomp_mod.set_param_hint('lamb',value=7)
result = gomp_mod.fit(y,t=t,verbose=False)
fixed_gomp[country]=result
### Fixed
gomp_dict['Benin']=fixed_gomp['Benin']

In [24]:
#### Senegal
country='Senegal'
y = np.asarray(dframe2.ix[country])
### Set hints from successful logistic fit
gomp_mod.set_param_hint('A',value=46,min=0,max=100)
gomp_mod.set_param_hint('mu',value=8,min=0)
gomp_mod.set_param_hint('lamb',value=5)
result = gomp_mod.fit(y,t=t,verbose=False)
fixed_gomp[country]=result
### Fixed
gomp_dict['Senegal'] = fixed_gomp['Senegal']

In [25]:
### Sudan
country='Sudan'
y = np.asarray(dframe2.ix[country])
### Set hints from successful logistic fit
gomp_mod.set_param_hint('A',value=90,min=0,max=100)
gomp_mod.set_param_hint('mu',value=7,min=0)
gomp_mod.set_param_hint('lamb',value=10)
result = gomp_mod.fit(y,t=t,verbose=False)
fixed_gomp[country]=result
### Fixed
gomp_dict['Sudan'] = fixed_gomp['Sudan']

In [26]:
### Bolivia
country='Bolivia'
y = np.asarray(dframe2.ix[country])
### Set hints from successful logistic fit
gomp_mod.set_param_hint('A',value=90,min=0,max=100)
gomp_mod.set_param_hint('mu',value=7,min=0)
gomp_mod.set_param_hint('lamb',value=5)
result = gomp_mod.fit(y,t=t,verbose=False)
fixed_gomp[country]=result
print(country,fixed_gomp['Bolivia'].best_values)
print('AIC:',fixed_gomp['Bolivia'].aic)
print('BIC:',fixed_gomp['Bolivia'].bic)

Bolivia {'lamb': 8.2197730719398407, 'A': 99.999999998836557, 'mu': 4.2466179149880716}
AIC: 16.2122420003
BIC: 18.3363926036


In [27]:
### Logistic fix
fixed_log = {}
bad_fits_log = ['Sao Tome and Principe','Sudan','Uganda']
for country in bad_fits_log:
    y = np.asarray(dframe2.ix[country])
    logmod.set_param_hint('A',value=dframe2.ix[country].max(),min=0,max=100)
    logmod.set_param_hint('mu',value=diff.ix[country].max(),min=0)
    logmod.set_param_hint('lamb',value=0)
    result = logmod.fit(y,t=t,verbose=False)
    fixed_log[country]=result

In [28]:
### Fixed: Uganda, Sao Tome and Principe
log_dict['Sao Tome and Principe'] = fixed_log['Sao Tome and Principe']
log_dict['Uganda'] = fixed_log['Uganda']

In [29]:
### Sudan
country='Sudan'
y = np.asarray(dframe2.ix[country])
logmod.set_param_hint('A',value=10,min=0,max=100)
logmod.set_param_hint('mu',value=1,min=0)
logmod.set_param_hint('lamb',value=2)
result = logmod.fit(y,t=t,verbose=False)
fixed_log[country] = result
### Fixed
log_dict['Sudan'] = fixed_log['Sudan']

In [30]:
### Algeria
country='Algeria'
y = np.asarray(dframe2.ix[country])
logmod.set_param_hint('A',value=90,min=0,max=100)
logmod.set_param_hint('mu',value=7,min=0)
logmod.set_param_hint('lamb',value=4)
result = logmod.fit(y,t=t,verbose=False)
fixed_log[country] = result
print(country,fixed_log['Algeria'].best_values)
print('AIC:',fixed_log['Algeria'].aic)
print('BIC:',fixed_log['Algeria'].bic)

Algeria {'lamb': 5.5771703767986791, 'A': 87.304516482746948, 'mu': 8.3298573584427462}
AIC: 55.2033409215
BIC: 57.3274915248


In [39]:
gomp_dict['Cuba'].plot_fit()
plt.show()

In [31]:
### Uzbekistan
country='Uzbekistan'
y = np.asarray(dframe2.ix[country])
logmod.set_param_hint('A',value=90,min=0,max=100)
logmod.set_param_hint('mu',value=7,min=0)
logmod.set_param_hint('lamb',value=8)
result = logmod.fit(y,t=t,verbose=False)
fixed_log[country] = result
print(country,fixed_log['Uzbekistan'].best_values)
print('AIC:',fixed_log['Uzbekistan'].aic)
print('BIC:',fixed_log['Uzbekistan'].bic)

Uzbekistan {'lamb': 10.764464167350875, 'A': 99.999999999028446, 'mu': 13.063134783685154}
AIC: -2.73431072151
BIC: -0.610160118199


In [32]:
country='Colombia'
y = np.asarray(dframe2.ix[country])
logmod.set_param_hint('A',value=90,min=0,max=100)
logmod.set_param_hint('mu',value=7,min=0)
logmod.set_param_hint('lamb',value=8)
result = logmod.fit(y,t=t,verbose=False)
fixed_log[country] = result
print(country,fixed_log['Colombia'].best_values)
print('AIC:',fixed_log['Colombia'].aic)
print('BIC:',fixed_log['Colombia'].bic)

Colombia {'lamb': 6.861379671055821, 'A': 99.999999860219461, 'mu': 6.9023989508828043}
AIC: 35.866067006
BIC: 37.9902176093


In [33]:
### Generate new figures 
fixed_plots = ['Benin','Chad','Senegal','Somalia','Sudan','Sao Tome and Principe','Uganda']
for country in fixed_plots:
    t2=np.arange(0,21)
    y=dframe2.ix[country]
    fig=plt.figure()
    plt.plot(t, y,         'bo')
    plt.plot(t2, gomp_dict[country].eval(t=t2), 'r-')
    plt.plot(t2, log_dict[country].eval(t=t2), 'g--')
    plt.title(country)
    plt.axis(x=np.arange(0,20))
    plt.ylabel('ART coverage (%)')
    plt.xlabel('Time (in years since 2000)')
    filename=country+'.png'
    plt.savefig(filename)

C:\Users\Ben\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [102]:
print('Gompertz')
for country in bad_fits_gomp:
    print(country,gomp_dict[country].best_values)
    print('AIC:',gomp_dict[country].aic)
    print('BIC:',gomp_dict[country].bic)
    
print('Logistic')
for country in bad_fits_log:
    print(country,log_dict[country].best_values)
    print('AIC:',log_dict[country].aic)
    print('BIC:',log_dict[country].bic)

Gompertz
Benin {'mu': 16.703609217725635, 'A': 43.873947820770162, 'lamb': 6.8622548899168798}
AIC: 35.7852413681
BIC: 37.9093919714
Chad {'mu': 3.7396198903530911, 'A': 32.158594086328662, 'lamb': 5.3075909887312331}
AIC: 11.653494487
BIC: 13.7776450903
Senegal {'mu': 7.5755889698460646, 'A': 42.973564458946775, 'lamb': 5.1955520209584094}
AIC: 41.9386123655
BIC: 44.0627629688
Somalia {'mu': 0.95709819474999391, 'A': 8.2779112494643652, 'lamb': 6.4929229663912347}
AIC: -30.3930914392
BIC: -28.2689408359
Sudan {'mu': 3.9832510373647692, 'A': 7.0975409279409094, 'lamb': 7.0912595467610995}
AIC: -7.87498528166
BIC: -5.75083467835
Logistic
Sao Tome and Principe {'mu': 7.6365416766888981, 'A': 68.866453467196791, 'lamb': 7.3252353605365981}
AIC: 10.079758167
BIC: 12.2039087703
Sudan {'mu': 5.0079749906210251, 'A': 6.9495658314029036, 'lamb': 7.2830464196349061}
AIC: -6.29178856067
BIC: -4.16763795737
Uganda {'mu': 8.1645654735134308, 'A': 99.999996780867122, 'lamb': 6.7747210995646281}
AIC

In [26]:
gomp_dict['Vietnam'].best_values

{'A': 56.192524710023108, 'lamb': 5.5434641721958986, 'mu': 6.264790720132293}

In [27]:
pop_data = pd.ExcelFile('Total_Population_2015.xlsx')
pop_frame = pop_data.parse('Data')
pop_frame.head()

,Country Name,Total Population (2015)
0,Afghanistan,32527000
1,Albania,2906000
2,Algeria,39667000
3,American Samoa,56000
4,Andorra,70000


In [28]:
pop_frame=pop_frame[pop_frame['Total Population (2015)']>1000000]

In [29]:
len(pop_frame)

191

In [30]:
estFrame = DataFrame(columns=['Country Name','A','lamb','mu','AIC','BIC'])

In [31]:
for country in gomp_dict:
    dic = gomp_dict[country].best_values
    dic['Country Name'] = country
    dic['AIC']=gomp_dict[country].aic
    dic['BIC']=gomp_dict[country].bic
    estFrame = estFrame.append(dic,ignore_index=True)

In [32]:
estFrame.head(10)

,Country Name,A,lamb,mu,AIC,BIC
0,Oman,99.999996,-9.324894,2.145867,63.461133,65.585284
1,Guinea,38.099388,5.349926,4.912033,13.150502,15.274653
2,Ghana,55.818384,6.381934,6.085414,9.783688,11.907839
3,South Africa,99.999987,7.394285,8.873088,37.127902,39.252053
4,Tunisia,28.884836,-0.544972,4.016785,23.463552,25.587703
5,Guinea-Bissau,29.398491,6.138490,3.448398,-15.437271,-13.313120
6,Arab World,50.629317,5.582012,2.342654,-25.995043,-23.870892
7,Sub-Saharan Africa (all income levels),99.999999,6.135956,6.523696,-3.748940,-1.624789
8,Guatemala,53.018204,1.879260,4.460883,35.177226,37.301377
9,Cuba,90.794056,-2.102193,6.436206,75.958529,78.082680


In [33]:
best_est = pd.merge(estFrame,pop_frame,how='inner',
                   left_on='Country Name',right_on='Country Name',sort=True)

In [34]:
best_est.head()

,Country Name,A,lamb,mu,AIC,BIC,Total Population (2015)
0,Afghanistan,5.066791,9.163537,1.351663,-42.037965,-39.913814,32527000
1,Algeria,100.000000,4.824685,7.469659,51.865712,53.989862,39667000
2,Angola,64.936576,6.438236,3.994152,10.362041,12.486192,25022000
3,Arab World,50.629317,5.582012,2.342654,-25.995043,-23.870892,393433000
4,Argentina,73.128153,-12.766129,2.312381,30.427524,32.551675,43417000


In [35]:
writer = pd.ExcelWriter('Gompertz_Estimates.xlsx')
best_est.to_excel(writer)
writer.save()

In [37]:
lmi = pd.ExcelFile('LMI_Countries.xlsx')
lmi = lmi.parse('Sheet1')

In [38]:
lmi_est = pd.merge(best_est,lmi,how='inner',
                   left_on='Country Name',right_on='Country Name',sort=True)

In [39]:
lmi_est.head()

,Country Name,A,lamb,mu,AIC,BIC,Total Population (2015),Code,Group Name,Country Code
0,Afghanistan,5.066791,9.163537,1.351663,-42.037965,-39.913814,32527000,LMY,Low & middle income,AFG
1,Algeria,100.000000,4.824685,7.469659,51.865712,53.989862,39667000,LMY,Low & middle income,DZA
2,Angola,64.936576,6.438236,3.994152,10.362041,12.486192,25022000,LMY,Low & middle income,AGO
3,Armenia,100.000000,9.228023,4.819737,-22.230641,-20.106491,3018000,LMY,Low & middle income,ARM
4,Azerbaijan,100.000000,10.129907,6.873259,-16.783587,-14.659436,9635000,LMY,Low & middle income,AZE


In [40]:
writer2 = pd.ExcelWriter('LMI_Estimates_Gompertz.xlsx')
lmi_est.to_excel(writer2)
writer2.save()

In [41]:
lmi_est['ART Coverage (2014)'] = dframe2[2014]

In [42]:
cov = DataFrame.from_dict(dframe2[2014]).reset_index(level=0)

In [43]:
cov_est = pd.merge(lmi_est,cov,how='inner',
                   left_on='Country Name',right_on='Country Name',sort=True)

In [44]:
cov_est.head()

,Country Name,A,lamb,mu,AIC,BIC,Total Population (2015),Code,Group Name,Country Code,ART Coverage (2014),2014.0
0,Afghanistan,5.066791,9.163537,1.351663,-42.037965,-39.913814,32527000,LMY,Low & middle income,AFG,NaN,5.00
1,Algeria,100.000000,4.824685,7.469659,51.865712,53.989862,39667000,LMY,Low & middle income,DZA,NaN,71.25
2,Angola,64.936576,6.438236,3.994152,10.362041,12.486192,25022000,LMY,Low & middle income,AGO,NaN,31.25
3,Armenia,100.000000,9.228023,4.819737,-22.230641,-20.106491,3018000,LMY,Low & middle income,ARM,NaN,23.75
4,Azerbaijan,100.000000,10.129907,6.873259,-16.783587,-14.659436,9635000,LMY,Low & middle income,AZE,NaN,27.50


In [45]:
cov_est.rename(columns={2014:'ART Coverage (2014)'},inplace=True)

In [47]:
writer3 = pd.ExcelWriter('Coverage and Estimates - Gompertz.xlsx')
cov_est.to_excel(writer3)
writer3.save()